In [3]:
# -*- coding: utf-8 -*-
"""
Scalability test — single-period InfoPool (fixed τ, 20 assets)
Scale N from 2 to 50 accounts.
Accounts 11..50 replicate info/constraints of 1..10 (pattern by n % 10).
No FairSplit / rebates here.

Outputs (in scalability_test_2_to_50/):
  - runtime_summary.csv   (N, build_time, solve_time, total_time, status)
  - runtime_vs_accounts.png
  - cached market data / matrices under _cache/ (pickle/npz, no pyarrow needed)
"""

import os
import time
import hashlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import xpress as xp


# -----------------------------
# 0) Paths & switches
# -----------------------------
OUTDIR = "scalability_test_2_to_50"
CACHE_DIR = os.path.join(OUTDIR, "_cache")
os.makedirs(OUTDIR, exist_ok=True)
os.makedirs(CACHE_DIR, exist_ok=True)

ANNUAL_TRADING_DAYS = 252

# -----------------------------
# Xpress init (guarded)
# -----------------------------
XPRESS_LICENSE_PATH = r"C:/Users/s2667242/AppData/Local/anaconda3/Lib/site-packages/xpress/license/xpauth.xpr"

def ensure_xpress_inited():
    if getattr(xp, "_JYH_ALREADY_INITED", False):
        return
    try:
        xp.init(XPRESS_LICENSE_PATH)
    except Exception as e:
        print("[warn] xp.init() failed or already initialized, proceeding if environment is live:", e)
    try:
        with xp.problem() as _p:
            _p.setObjective(1.0, sense=xp.maximize)
    except Exception as e:
        raise RuntimeError("Xpress global environment not available; restart kernel or fix license path.") from e
    xp._JYH_ALREADY_INITED = True

ensure_xpress_inited()

# -----------------------------
# 1) Market data & Ω-construction  (with pickle cache)
# -----------------------------
tickers = [
    "AAPL","MSFT","GOOGL","AMZN","META","TSLA","JNJ","JPM","V","PG",
    "NVDA","DIS","UNH","HD","MA","PFE","BAC","KO","XOM","CVX"
]
d = len(tickers)

start_date = "2020-01-01"
end_date   = "2025-07-24"

prices_adj_pkl = os.path.join(CACHE_DIR, "prices_adj.pkl")
prices_raw_pkl = os.path.join(CACHE_DIR, "prices_raw.pkl")
vols_pkl       = os.path.join(CACHE_DIR, "volumes.pkl")
omega_npz      = os.path.join(CACHE_DIR, "omegas_corr.npz")

def winsorize_ser(s: pd.Series, q=0.995):
    lo, hi = s.quantile(1-q), s.quantile(q)
    return s.clip(lower=lo, upper=hi)

def estimate_kappa_from_illiq(prices_raw, volumes, returns, sigma_vec, side="both"):
    px_raw = prices_raw.loc[returns.index]
    vol    = volumes.loc[returns.index]
    dollar_vol = (px_raw * vol).replace(0, np.nan)
    illiq = (returns.abs() / dollar_vol).replace([np.inf, -np.inf], np.nan)
    if side == "plus":
        mask = (returns > 0)
    elif side == "minus":
        mask = (returns < 0)
    else:
        mask = pd.DataFrame(True, index=returns.index, columns=returns.columns)
    illiq_med = winsorize_ser(illiq.where(mask).median(axis=0, skipna=True), q=0.995)
    Vbar_local = (px_raw * vol).mean(axis=0)
    sigma = pd.Series(sigma_vec, index=returns.columns)
    kappa = (illiq_med * Vbar_local / sigma).replace([np.inf, -np.inf], np.nan)
    return kappa.clip(lower=0.01, upper=100.0).values

def gershgorin_delta_max(gamma: np.ndarray, corr: np.ndarray):
    G = np.sqrt(np.outer(gamma, gamma))
    A = np.abs(corr) * G
    row_sum_off = A.sum(axis=1) - np.diag(A)
    row_sum_off = np.where(row_sum_off <= 1e-12, np.inf, row_sum_off)
    return float(np.min(gamma / row_sum_off))

def delta_from_ratio(gamma: np.ndarray, corr: np.ndarray, rho=0.3, cap_ratio=0.9):
    G = np.sqrt(np.outer(gamma, gamma))
    A = np.abs(corr) * G
    row_sum_off = A.sum(axis=1) - np.diag(A)
    factor = row_sum_off / gamma
    mf = np.mean(factor[np.isfinite(factor) & (factor > 0)])
    delta_star = rho / mf if (mf is not None and mf > 0) else 0.0
    delta_max  = gershgorin_delta_max(gamma, corr)
    return float(min(delta_star, cap_ratio * delta_max)), float(delta_max)

print("[info] Preparing market data and impact matrices (pickle cache) ...")
need_download = not (os.path.exists(prices_adj_pkl) and os.path.exists(prices_raw_pkl) and os.path.exists(vols_pkl))
if need_download:
    data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=False, progress=False)
    prices_adj = data["Adj Close"]
    prices_raw = data["Close"]
    volumes    = data["Volume"]
    prices_adj.to_pickle(prices_adj_pkl)
    prices_raw.to_pickle(prices_raw_pkl)
    volumes.to_pickle(vols_pkl)
else:
    prices_adj = pd.read_pickle(prices_adj_pkl)
    prices_raw = pd.read_pickle(prices_raw_pkl)
    volumes    = pd.read_pickle(vols_pkl)

returns = prices_adj.pct_change().dropna()
mean_daily_returns = returns.mean()
cov_matrix         = returns.cov()

sigma_all   = returns.std(axis=0).values
up_returns  = returns.clip(lower=0)
down_returns= (-returns).clip(lower=0)
sigma_plus  = up_returns.std(axis=0).values
sigma_minus = down_returns.std(axis=0).values

dollar_vol = (prices_raw.reindex_like(volumes) * volumes).loc[returns.index]
Vbar = dollar_vol.mean(axis=0).values

if os.path.exists(omega_npz):
    npz = np.load(omega_npz)
    Omega_plus  = npz["Op"]
    Omega_minus = npz["Om"]
    corr        = npz["corr"]
else:
    corr = cov_matrix.values / np.outer(sigma_all, sigma_all)
    kappa_plus_hat  = estimate_kappa_from_illiq(prices_raw, volumes, returns, sigma_plus,  side="plus")
    kappa_minus_hat = estimate_kappa_from_illiq(prices_raw, volumes, returns, sigma_minus, side="minus")
    gamma_plus  = kappa_plus_hat  * sigma_plus  / Vbar
    gamma_minus = kappa_minus_hat * sigma_minus / Vbar
    delta_plus,  _ = delta_from_ratio(gamma_plus,  corr, rho=0.3, cap_ratio=0.9)
    delta_minus, _ = delta_from_ratio(gamma_minus, corr, rho=0.3, cap_ratio=0.9)
    Gp = np.sqrt(np.outer(gamma_plus,  gamma_plus))
    Gm = np.sqrt(np.outer(gamma_minus, gamma_minus))
    Omega_plus  = np.diag(gamma_plus)  + delta_plus  * (Gp * corr)
    Omega_minus = np.diag(gamma_minus) + delta_minus * (Gm * corr)
    # scale to target bps
    xi = 0.10
    target_bps = 100
    c = target_bps / 1e4  # 100 bps = 0.01
    s_cand_p = 2*c / np.maximum(gamma_plus  * xi * Vbar, 1e-30)
    s_cand_m = 2*c / np.maximum(gamma_minus * xi * Vbar, 1e-30)
    s_omega  = float(np.median(np.concatenate([s_cand_p, s_cand_m])))
    Omega_plus  = s_omega * Omega_plus
    Omega_minus = s_omega * Omega_minus
    np.savez_compressed(omega_npz, Op=Omega_plus, Om=Omega_minus, corr=corr)

for name, M in [("Omega_plus", Omega_plus), ("Omega_minus", Omega_minus)]:
    if not np.isfinite(M).all():
        raise ValueError(f"{name} contains non-finite values.")

# -----------------------------
# 2) Build up to 50 accounts by replicating the first 10
# -----------------------------
N_MAX = 50     # scale to 50
BASE = 10      # replicate every 10 accounts (pattern by n % 10)

gamma_core = [
    0.49671415, 0.64768854, 0.76743473, 0.54256004, 0.24196227,
    0.31424733, 0.06752820, 0.11092259, 0.37569802, 0.82254491
]
budgets_core = [1e6, 5e5, 8e5, 1.2e6, 6e5, 9e5, 7e5, 4e5, 1.5e6, 1e6]

# repeat core pattern to length 50
repeat_times = (N_MAX + BASE - 1) // BASE  # ceil(50/10)=5
gamma_all   = (gamma_core   * repeat_times)[:N_MAX]
budgets_all = (budgets_core * repeat_times)[:N_MAX]

trading_days = ANNUAL_TRADING_DAYS
mu_vec      = mean_daily_returns.values
Sigma_mat   = cov_matrix.values
mu_annual   = mu_vec * trading_days
Sigma_annual= Sigma_mat * trading_days

mu_list_ALL    = [mu_annual.copy()    for _ in range(N_MAX)]
Sigma_list_ALL = [Sigma_annual.copy() for _ in range(N_MAX)]

# risk scaling θ (pattern replicated; common scale keeps pattern invariant)
w_pct0 = np.full((N_MAX, d), 1.0/d)
risk_proto = np.array([
    0.5 * gamma_all[n] * (budgets_all[n]**2) * (w_pct0[n] @ Sigma_list_ALL[n] @ w_pct0[n])
    for n in range(N_MAX)
])
rev_est = np.array([budgets_all[n] * (mu_list_ALL[n] @ w_pct0[n]) for n in range(N_MAX)])
alpha_risk = 0.5
s_theta = (alpha_risk * max(rev_est.mean(), 1e-12)) / max(risk_proto.mean(), 1e-12)
theta_list_ALL = (s_theta * np.array(gamma_all)).tolist()

# initial holdings: build deterministic base-10 and replicate to 50
INIT_HOLDINGS_PATH = os.path.join(CACHE_DIR, "initial_holdings_50rep.csv")
if os.path.exists(INIT_HOLDINGS_PATH):
    old_holdings_ALL = pd.read_csv(INIT_HOLDINGS_PATH, index_col=0).values
    if old_holdings_ALL.shape != (N_MAX, d):
        os.remove(INIT_HOLDINGS_PATH)
        old_holdings_ALL = None
else:
    old_holdings_ALL = None

if old_holdings_ALL is None:
    def _dirichlet_like_from_hash(tickers, budgets, seed=12345):
        Nn, dd = len(budgets), len(tickers)
        W = np.zeros((Nn, dd), dtype=float)
        for n in range(Nn):
            e = np.empty(dd, dtype=float)
            for j, t in enumerate(tickers):
                key = f"{seed}|acct={n}|ticker={t}".encode()
                h = hashlib.sha256(key).digest()
                u64 = int.from_bytes(h[:8], "big")
                u = (u64 + 0.5) / 2**64
                u = min(max(u, 1e-12), 1-1e-12)
                e[j] = -np.log(u)
            w = e / e.sum()
            W[n, :] = w
        B = np.asarray(budgets, dtype=float)[:, None]
        return W * B

    base10 = _dirichlet_like_from_hash(tickers, budgets_core, seed=12345)  # (10×20)
    old_holdings_ALL = np.vstack([base10 for _ in range(repeat_times)])[:N_MAX, :]  # (50×20)
    pd.DataFrame(old_holdings_ALL,
                 index=[f"Acct {i}" for i in range(N_MAX)],
                 columns=tickers).to_csv(INIT_HOLDINGS_PATH, float_format="%.6f")

# -----------------------------
# 3) Fixed τ (N×20): Acct 0 never shares; others share
# -----------------------------
TAU_BASE = np.ones((N_MAX, d), dtype=int)
TAU_BASE[0, :] = 0
pd.DataFrame(TAU_BASE, index=[f"Acct {i}" for i in range(N_MAX)], columns=tickers)\
  .to_csv(os.path.join(OUTDIR, "tau_base_used.csv"), float_format="%.0f")

# -----------------------------
# 4) Constraints replicator: account n uses pattern of (n % 10)
# -----------------------------
SECTOR_IDXS = {
    "T":  [tickers.index(t) for t in ["AAPL","MSFT","NVDA"]],
    "C":  [tickers.index(t) for t in ["GOOGL","META","DIS"]],
    "R":  [tickers.index(t) for t in ["AMZN","HD","TSLA"]],
    "H":  [tickers.index(t) for t in ["JNJ","UNH","PFE"]],
    "F":  [tickers.index(t) for t in ["JPM","BAC","V","MA"]],
    "S":  [tickers.index(t) for t in ["PG","KO"]],
    "E":  [tickers.index(t) for t in ["XOM","CVX"]],
}
GROWTH8 = [tickers.index(t) for t in ["NVDA","TSLA","AAPL","MSFT"]]

def add_constraints_for_account(model, n, w, budgets):
    p = n % 10
    if p == 0:
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["S"]+SECTOR_IDXS["H"]) >=  budgets[n]*0.40)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["T"]+SECTOR_IDXS["R"]) <=  budgets[n]*0.25)
    elif p == 1:
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["T"]) >=  budgets[n]*0.40)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["C"]+[tickers.index("AMZN")]) >= budgets[n]*0.30)
    elif p == 2:
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["T"]) >=  budgets[n]*0.10)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["T"]) <=  budgets[n]*0.25)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["C"]) >=  budgets[n]*0.10)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["C"]) <=  budgets[n]*0.25)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["H"]) >=  budgets[n]*0.10)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["H"]) <=  budgets[n]*0.25)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["F"]) >=  budgets[n]*0.10)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["F"]) <=  budgets[n]*0.25)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["S"]+SECTOR_IDXS["E"]) >= budgets[n]*0.05)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["S"]+SECTOR_IDXS["E"]) <= budgets[n]*0.15)
    elif p == 3:
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["S"]+SECTOR_IDXS["F"]+SECTOR_IDXS["E"]) >= budgets[n]*0.50)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["F"]) >=  budgets[n]*0.20)
    elif p == 4:
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["E"]) >=  budgets[n]*0.15)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["F"]) >=  budgets[n]*0.25)
    elif p == 5:
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["T"]) >=  budgets[n]*0.30)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["H"]) >=  budgets[n]*0.20)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["S"]+SECTOR_IDXS["R"]) <= budgets[n]*0.15)
    elif p == 6:
        growth6 = SECTOR_IDXS["T"] + SECTOR_IDXS["C"] + SECTOR_IDXS["R"]
        model.addConstraint(xp.Sum(w[n][j] for j in growth6) <= budgets[n]*0.35)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["E"]+SECTOR_IDXS["F"]+SECTOR_IDXS["S"]) >= budgets[n]*0.40)
    elif p == 7:
        model.addConstraint(xp.Sum(w[n][j] for j in [tickers.index("AMZN"), tickers.index("TSLA")]) >= budgets[n]*0.30)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["T"]+SECTOR_IDXS["F"]+SECTOR_IDXS["H"]) <= budgets[n]*0.50)
    elif p == 8:
        model.addConstraint(xp.Sum(w[n][j] for j in GROWTH8) >= budgets[n]*0.50)
    elif p == 9:
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["S"]+SECTOR_IDXS["E"]) >= budgets[n]*0.20)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["T"]+SECTOR_IDXS["C"]+SECTOR_IDXS["R"]) >= budgets[n]*0.30)

# -----------------------------
# 5) Core solver (build-time & solve-time separated)
# -----------------------------
def solve_infopool_once(N: int, tau_mat: np.ndarray,
                        budgets, mu_list, Sigma_list,
                        Omega_plus, Omega_minus,
                        old_holdings, theta_list):
    assert tau_mat.shape == (N, d)
    assert old_holdings.shape == (N, d)

    with xp.problem() as model:
        model.controls.outputlog = 0  # faster; comment out to see logs
        # model.controls.threads = -1  # optional: all threads

        tb0 = time.perf_counter()

        # variables
        w   = [[model.addVariable(lb=0.0, ub=budgets[n]) for j in range(d)] for n in range(N)]
        wp  = [[model.addVariable(lb=0.0, ub=budgets[n]) for j in range(d)] for n in range(N)]
        wm  = [[model.addVariable(lb=0.0, ub=budgets[n]) for j in range(d)] for n in range(N)]
        w_pct = [[model.addVariable(lb=0.0, ub=1.0)     for j in range(d)] for n in range(N)]

        # buy/sell split
        for n in range(N):
            for j in range(d):
                model.addConstraint(wp[n][j] - wm[n][j] == w[n][j] - old_holdings[n][j])

        # budgets + simplex
        for n in range(N):
            model.addConstraint(xp.Sum(w[n][j] for j in range(d)) == budgets[n])
            for j in range(d):
                model.addConstraint(w[n][j] == budgets[n] * w_pct[n][j])
            model.addConstraint(xp.Sum(w_pct[n][j] for j in range(d)) == 1)

        # replicate constraints by pattern p = n % 10
        for n in range(N):
            add_constraints_for_account(model, n, w, budgets)

        # objective
        lin = xp.Sum(mu_list[n][j] * w[n][j] for n in range(N) for j in range(d))
        risk = xp.Sum(
            -0.5 * theta_list[n] * (budgets[n]**2) *
            xp.Sum(Sigma_list[n][i][j] * w_pct[n][i] * w_pct[n][j] for i in range(d) for j in range(d))
            for n in range(N)
        )

        tau_arr = np.asarray(tau_mat, dtype=int)
        T_plus  = [xp.Sum(tau_arr[k, j] * wp[k][j]  for k in range(N)) for j in range(d)]
        T_minus = [xp.Sum(tau_arr[k, j] * wm[k][j] for k in range(N)) for j in range(d)]

        quad_pp = xp.Sum(T_plus[i]  * xp.Sum(Omega_plus[i,  j] * T_plus[j]  for j in range(d)) for i in range(d))
        quad_mm = xp.Sum(T_minus[i] * xp.Sum(Omega_plus[i,  j] * T_minus[j] for j in range(d)) for i in range(d))
        cross_pm= xp.Sum(T_plus[i]  * xp.Sum(Omega_minus[i, j] * T_minus[j] for j in range(d)) for i in range(d))
        impact_pool = 0.5 * (quad_pp + quad_mm + 2 * cross_pm)

        impact_ind = 0
        for m in range(N):
            r_p = [ (1 - tau_arr[m, j]) * wp[m][j]  for j in range(d) ]
            r_m = [ (1 - tau_arr[m, j]) * wm[m][j] for j in range(d) ]
            quad_pp_m = xp.Sum(r_p[i] * xp.Sum(Omega_plus[i,  j] * r_p[j] for j in range(d)) for i in range(d))
            quad_mm_m = xp.Sum(r_m[i] * xp.Sum(Omega_plus[i,  j] * r_m[j] for j in range(d)) for i in range(d))
            cross_pm_m= xp.Sum(r_p[i] * xp.Sum(Omega_minus[i, j] * r_m[j] for j in range(d)) for i in range(d))
            impact_ind += 0.5 * (quad_pp_m + quad_mm_m + 2.0 * cross_pm_m)

        impact = - ANNUAL_TRADING_DAYS * (impact_pool + impact_ind)
        model.setObjective(lin + risk + impact, sense=xp.maximize)

        tb1 = time.perf_counter()
        model.solve()
        ts1 = time.perf_counter()

        build_time = tb1 - tb0
        solve_time = ts1 - tb1

    return build_time, solve_time

# -----------------------------
# 6) Run scalability sweep: N = 2..50
# -----------------------------
runtime_records = []
print("\n[info] Starting scalability sweep (N = 2..50) ...")

for N in range(2, N_MAX+1):
    print(f"[run ] N = {N} accounts ...")

    budgets     = budgets_all[:N]
    mu_list     = mu_list_ALL[:N]
    Sigma_list  = Sigma_list_ALL[:N]
    theta_list  = theta_list_ALL[:N]
    old_holdings= old_holdings_ALL[:N, :]
    tau_N       = TAU_BASE[:N, :].copy()

    try:
        build_t, solve_t = solve_infopool_once(
            N, tau_N, budgets, mu_list, Sigma_list,
            Omega_plus, Omega_minus, old_holdings, theta_list
        )
        total_t = build_t + solve_t
        status = "ok"
    except Exception as e:
        build_t = np.nan; solve_t = np.nan; total_t = np.nan
        status = f"error: {e}"
        print(f"[ERR ] N={N}: {e}")

    runtime_records.append({
        "N": N,
        "build_time_sec": build_t,
        "solve_time_sec": solve_t,
        "total_time_sec": total_t,
        "status": status
    })
    print(f"[done] N={N}: build={build_t:.4f}s, solve={solve_t:.4f}s, total={total_t:.4f}s, status={status}")

# Master runtime index
rt_df = pd.DataFrame(runtime_records)
rt_path = os.path.join(OUTDIR, "runtime_summary.csv")
rt_df.to_csv(rt_path, index=False)
print("\n[info] Runtime summary saved to:", rt_path)

# -----------------------------
# 7) Visualization
# -----------------------------
ok_df = rt_df[rt_df["status"] == "ok"].copy()
plt.figure(figsize=(7.2, 4.4))
plt.plot(ok_df["N"], ok_df["total_time_sec"], marker="o")
plt.xlabel("Number of Accounts (N)")
plt.ylabel("Total Time (seconds)")
plt.title("InfoPool Single-Period Scalability (Assets = 20): N=2..50")
plt.grid(True, linestyle="--", alpha=0.5)
png_path = os.path.join(OUTDIR, "runtime_vs_accounts.png")
plt.tight_layout()
plt.savefig(png_path, dpi=180)
plt.close()
print("[info] Plot saved to:", png_path)

print("\nAll scalability tests finished.")
print(f"Outputs are in: {os.path.abspath(OUTDIR)}")


[info] Preparing market data and impact matrices (pickle cache) ...

[info] Starting scalability sweep (N = 2..50) ...
[run ] N = 2 accounts ...
[done] N=2: build=0.0503s, solve=0.0074s, total=0.0577s, status=ok
[run ] N = 3 accounts ...
[done] N=3: build=0.0651s, solve=0.0102s, total=0.0754s, status=ok
[run ] N = 4 accounts ...
[done] N=4: build=0.0648s, solve=0.0124s, total=0.0772s, status=ok
[run ] N = 5 accounts ...
[done] N=5: build=0.1191s, solve=0.0161s, total=0.1353s, status=ok
[run ] N = 6 accounts ...
[done] N=6: build=0.1630s, solve=0.0230s, total=0.1860s, status=ok
[run ] N = 7 accounts ...
[done] N=7: build=0.1607s, solve=0.0264s, total=0.1871s, status=ok
[run ] N = 8 accounts ...
[done] N=8: build=0.1809s, solve=0.0276s, total=0.2085s, status=ok
[run ] N = 9 accounts ...
[done] N=9: build=0.1937s, solve=0.0391s, total=0.2328s, status=ok
[run ] N = 10 accounts ...
[done] N=10: build=0.2517s, solve=0.0404s, total=0.2921s, status=ok
[run ] N = 11 accounts ...
[done] N=11: bu

In [4]:
# -*- coding: utf-8 -*-
"""
Scalability test — single-period InfoPool (fixed τ, 20 assets) with uncertainty bands
Scale N from 2 to 50 accounts; replicate accounts 11..50 from 1..10 (pattern n % 10).
Add K repeated runs per N via small Dirichlet noise on initial holdings to quantify uncertainty.

Outputs (in scalability_test_2_to_50/):
  - runtime_repeats_long.csv   (N, rep, build_time, solve_time, total_time, status)
  - runtime_summary.csv        (N, mean, std, ci_low, ci_high, slope_info)
  - runtime_vs_accounts_CI.png (mean curve with 95% CI ribbon)
  - loglog_fit.png             (log-log fit of time vs N with slope & 95% CI)
  - _cache/  (pickle/npz for market data & Ω matrices; no pyarrow needed)
"""

import os
import time
import hashlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import xpress as xp

# -----------------------------
# 0) Paths & switches
# -----------------------------
OUTDIR = "scalability_test_2_to_50"
CACHE_DIR = os.path.join(OUTDIR, "_cache")
os.makedirs(OUTDIR, exist_ok=True)
os.makedirs(CACHE_DIR, exist_ok=True)

ANNUAL_TRADING_DAYS = 252
REPEATS = 10                
DIRICHLET_ALPHA = 200.0      
RANDOM_SEED = 20250101      

# -----------------------------
# Xpress init (guarded)
# -----------------------------
XPRESS_LICENSE_PATH = r"C:/Users/s2667242/AppData/Local/anaconda3/Lib/site-packages/xpress/license/xpauth.xpr"

def ensure_xpress_inited():
    if getattr(xp, "_JYH_ALREADY_INITED", False):
        return
    try:
        xp.init(XPRESS_LICENSE_PATH)
    except Exception as e:
        print("[warn] xp.init() failed or already initialized, proceeding if environment is live:", e)
    try:
        with xp.problem() as _p:
            _p.setObjective(1.0, sense=xp.maximize)
    except Exception as e:
        raise RuntimeError("Xpress global environment not available; restart kernel or fix license path.") from e
    xp._JYH_ALREADY_INITED = True

ensure_xpress_inited()

# -----------------------------
# 1) Market data & Ω-construction  (with pickle cache)
# -----------------------------
tickers = [
    "AAPL","MSFT","GOOGL","AMZN","META","TSLA","JNJ","JPM","V","PG",
    "NVDA","DIS","UNH","HD","MA","PFE","BAC","KO","XOM","CVX"
]
d = len(tickers)

start_date = "2020-01-01"
end_date   = "2025-07-24"

prices_adj_pkl = os.path.join(CACHE_DIR, "prices_adj.pkl")
prices_raw_pkl = os.path.join(CACHE_DIR, "prices_raw.pkl")
vols_pkl       = os.path.join(CACHE_DIR, "volumes.pkl")
omega_npz      = os.path.join(CACHE_DIR, "omegas_corr.npz")

def winsorize_ser(s: pd.Series, q=0.995):
    lo, hi = s.quantile(1-q), s.quantile(q)
    return s.clip(lower=lo, upper=hi)

def estimate_kappa_from_illiq(prices_raw, volumes, returns, sigma_vec, side="both"):
    px_raw = prices_raw.loc[returns.index]
    vol    = volumes.loc[returns.index]
    dollar_vol = (px_raw * vol).replace(0, np.nan)
    illiq = (returns.abs() / dollar_vol).replace([np.inf, -np.inf], np.nan)
    if side == "plus":
        mask = (returns > 0)
    elif side == "minus":
        mask = (returns < 0)
    else:
        mask = pd.DataFrame(True, index=returns.index, columns=returns.columns)
    illiq_med = winsorize_ser(illiq.where(mask).median(axis=0, skipna=True), q=0.995)
    Vbar_local = (px_raw * vol).mean(axis=0)
    sigma = pd.Series(sigma_vec, index=returns.columns)
    kappa = (illiq_med * Vbar_local / sigma).replace([np.inf, -np.inf], np.nan)
    return kappa.clip(lower=0.01, upper=100.0).values

def gershgorin_delta_max(gamma: np.ndarray, corr: np.ndarray):
    G = np.sqrt(np.outer(gamma, gamma))
    A = np.abs(corr) * G
    row_sum_off = A.sum(axis=1) - np.diag(A)
    row_sum_off = np.where(row_sum_off <= 1e-12, np.inf, row_sum_off)
    return float(np.min(gamma / row_sum_off))

def delta_from_ratio(gamma: np.ndarray, corr: np.ndarray, rho=0.3, cap_ratio=0.9):
    G = np.sqrt(np.outer(gamma, gamma))
    A = np.abs(corr) * G
    row_sum_off = A.sum(axis=1) - np.diag(A)
    factor = row_sum_off / gamma
    mf = np.mean(factor[np.isfinite(factor) & (factor > 0)])
    delta_star = rho / mf if (mf is not None and mf > 0) else 0.0
    delta_max  = gershgorin_delta_max(gamma, corr)
    return float(min(delta_star, cap_ratio * delta_max)), float(delta_max)

print("[info] Preparing market data and impact matrices (pickle cache) ...")
need_download = not (os.path.exists(prices_adj_pkl) and os.path.exists(prices_raw_pkl) and os.path.exists(vols_pkl))
if need_download:
    data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=False, progress=False)
    prices_adj = data["Adj Close"]
    prices_raw = data["Close"]
    volumes    = data["Volume"]
    prices_adj.to_pickle(prices_adj_pkl)
    prices_raw.to_pickle(prices_raw_pkl)
    volumes.to_pickle(vols_pkl)
else:
    prices_adj = pd.read_pickle(prices_adj_pkl)
    prices_raw = pd.read_pickle(prices_raw_pkl)
    volumes    = pd.read_pickle(vols_pkl)

returns = prices_adj.pct_change().dropna()
mean_daily_returns = returns.mean()
cov_matrix         = returns.cov()

sigma_all   = returns.std(axis=0).values
up_returns  = returns.clip(lower=0)
down_returns= (-returns).clip(lower=0)
sigma_plus  = up_returns.std(axis=0).values
sigma_minus = down_returns.std(axis=0).values

dollar_vol = (prices_raw.reindex_like(volumes) * volumes).loc[returns.index]
Vbar = dollar_vol.mean(axis=0).values

if os.path.exists(omega_npz):
    npz = np.load(omega_npz)
    Omega_plus  = npz["Op"]
    Omega_minus = npz["Om"]
    corr        = npz["corr"]
else:
    corr = cov_matrix.values / np.outer(sigma_all, sigma_all)
    kappa_plus_hat  = estimate_kappa_from_illiq(prices_raw, volumes, returns, sigma_plus,  side="plus")
    kappa_minus_hat = estimate_kappa_from_illiq(prices_raw, volumes, returns, sigma_minus, side="minus")
    gamma_plus  = kappa_plus_hat  * sigma_plus  / Vbar
    gamma_minus = kappa_minus_hat * sigma_minus / Vbar
    delta_plus,  _ = delta_from_ratio(gamma_plus,  corr, rho=0.3, cap_ratio=0.9)
    delta_minus, _ = delta_from_ratio(gamma_minus, corr, rho=0.3, cap_ratio=0.9)
    Gp = np.sqrt(np.outer(gamma_plus,  gamma_plus))
    Gm = np.sqrt(np.outer(gamma_minus, gamma_minus))
    Omega_plus  = np.diag(gamma_plus)  + delta_plus  * (Gp * corr)
    Omega_minus = np.diag(gamma_minus) + delta_minus * (Gm * corr)
    # scale to target bps
    xi = 0.10
    target_bps = 100
    c = target_bps / 1e4  # 100 bps = 0.01
    s_cand_p = 2*c / np.maximum(gamma_plus  * xi * Vbar, 1e-30)
    s_cand_m = 2*c / np.maximum(gamma_minus * xi * Vbar, 1e-30)
    s_omega  = float(np.median(np.concatenate([s_cand_p, s_cand_m])))
    Omega_plus  = s_omega * Omega_plus
    Omega_minus = s_omega * Omega_minus
    np.savez_compressed(omega_npz, Op=Omega_plus, Om=Omega_minus, corr=corr)

for name, M in [("Omega_plus", Omega_plus), ("Omega_minus", Omega_minus)]:
    if not np.isfinite(M).all():
        raise ValueError(f"{name} contains non-finite values.")

# -----------------------------
# 2) Build up to 50 accounts by replicating the first 10
# -----------------------------
N_MAX = 50     # scale to 50
BASE = 10      # replicate every 10 accounts (pattern by n % 10)

gamma_core = [
    0.49671415, 0.64768854, 0.76743473, 0.54256004, 0.24196227,
    0.31424733, 0.06752820, 0.11092259, 0.37569802, 0.82254491
]
budgets_core = [1e6, 5e5, 8e5, 1.2e6, 6e5, 9e5, 7e5, 4e5, 1.5e6, 1e6]

repeat_times = (N_MAX + BASE - 1) // BASE  # ceil(50/10)=5
gamma_all   = (gamma_core   * repeat_times)[:N_MAX]
budgets_all = (budgets_core * repeat_times)[:N_MAX]

trading_days = ANNUAL_TRADING_DAYS
mu_vec      = mean_daily_returns.values
Sigma_mat   = cov_matrix.values
mu_annual   = mu_vec * trading_days
Sigma_annual= Sigma_mat * trading_days

mu_list_ALL    = [mu_annual.copy()    for _ in range(N_MAX)]
Sigma_list_ALL = [Sigma_annual.copy() for _ in range(N_MAX)]

# risk scaling θ
w_pct0 = np.full((N_MAX, d), 1.0/d)
risk_proto = np.array([
    0.5 * gamma_all[n] * (budgets_all[n]**2) * (w_pct0[n] @ Sigma_list_ALL[n] @ w_pct0[n])
    for n in range(N_MAX)
])
rev_est = np.array([budgets_all[n] * (mu_list_ALL[n] @ w_pct0[n]) for n in range(N_MAX)])
alpha_risk = 0.5
s_theta = (alpha_risk * max(rev_est.mean(), 1e-12)) / max(risk_proto.mean(), 1e-12)
theta_list_ALL = (s_theta * np.array(gamma_all)).tolist()

# initial holdings: deterministic base-10 and replicate to 50
INIT_HOLDINGS_PATH = os.path.join(CACHE_DIR, "initial_holdings_50rep.csv")
if os.path.exists(INIT_HOLDINGS_PATH):
    old_holdings_ALL = pd.read_csv(INIT_HOLDINGS_PATH, index_col=0).values
    if old_holdings_ALL.shape != (N_MAX, d):
        os.remove(INIT_HOLDINGS_PATH)
        old_holdings_ALL = None
else:
    old_holdings_ALL = None

def _dirichlet_like_from_hash(tickers, budgets, seed=12345):
    Nn, dd = len(budgets), len(tickers)
    W = np.zeros((Nn, dd), dtype=float)
    for n in range(Nn):
        e = np.empty(dd, dtype=float)
        for j, t in enumerate(tickers):
            key = f"{seed}|acct={n}|ticker={t}".encode()
            h = hashlib.sha256(key).digest()
            u64 = int.from_bytes(h[:8], "big")
            u = (u64 + 0.5) / 2**64
            u = min(max(u, 1e-12), 1-1e-12)
            e[j] = -np.log(u)
        w = e / e.sum()
        W[n, :] = w
    B = np.asarray(budgets, dtype=float)[:, None]
    return W * B

if old_holdings_ALL is None:
    base10 = _dirichlet_like_from_hash(tickers, budgets_core, seed=12345)  # (10×20)
    old_holdings_ALL = np.vstack([base10 for _ in range(repeat_times)])[:N_MAX, :]  # (50×20)
    pd.DataFrame(old_holdings_ALL,
                 index=[f"Acct {i}" for i in range(N_MAX)],
                 columns=tickers).to_csv(INIT_HOLDINGS_PATH, float_format="%.6f")

# -----------------------------
# 3) Fixed τ (N×20): Acct 0 never shares; others share
# -----------------------------
TAU_BASE = np.ones((N_MAX, d), dtype=int)
TAU_BASE[0, :] = 0
pd.DataFrame(TAU_BASE, index=[f"Acct {i}" for i in range(N_MAX)], columns=tickers)\
  .to_csv(os.path.join(OUTDIR, "tau_base_used.csv"), float_format="%.0f")

# -----------------------------
# 4) Constraints replicator: account n uses pattern of (n % 10)
# -----------------------------
SECTOR_IDXS = {
    "T":  [tickers.index(t) for t in ["AAPL","MSFT","NVDA"]],
    "C":  [tickers.index(t) for t in ["GOOGL","META","DIS"]],
    "R":  [tickers.index(t) for t in ["AMZN","HD","TSLA"]],
    "H":  [tickers.index(t) for t in ["JNJ","UNH","PFE"]],
    "F":  [tickers.index(t) for t in ["JPM","BAC","V","MA"]],
    "S":  [tickers.index(t) for t in ["PG","KO"]],
    "E":  [tickers.index(t) for t in ["XOM","CVX"]],
}
GROWTH8 = [tickers.index(t) for t in ["NVDA","TSLA","AAPL","MSFT"]]

def add_constraints_for_account(model, n, w, budgets):
    p = n % 10
    if p == 0:
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["S"]+SECTOR_IDXS["H"]) >=  budgets[n]*0.40)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["T"]+SECTOR_IDXS["R"]) <=  budgets[n]*0.25)
    elif p == 1:
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["T"]) >=  budgets[n]*0.40)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["C"]+[tickers.index("AMZN")]) >= budgets[n]*0.30)
    elif p == 2:
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["T"]) >=  budgets[n]*0.10)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["T"]) <=  budgets[n]*0.25)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["C"]) >=  budgets[n]*0.10)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["C"]) <=  budgets[n]*0.25)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["H"]) >=  budgets[n]*0.10)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["H"]) <=  budgets[n]*0.25)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["F"]) >=  budgets[n]*0.10)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["F"]) <=  budgets[n]*0.25)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["S"]+SECTOR_IDXS["E"]) >= budgets[n]*0.05)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["S"]+SECTOR_IDXS["E"]) <= budgets[n]*0.15)
    elif p == 3:
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["S"]+SECTOR_IDXS["F"]+SECTOR_IDXS["E"]) >= budgets[n]*0.50)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["F"]) >=  budgets[n]*0.20)
    elif p == 4:
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["E"]) >=  budgets[n]*0.15)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["F"]) >=  budgets[n]*0.25)
    elif p == 5:
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["T"]) >=  budgets[n]*0.30)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["H"]) >=  budgets[n]*0.20)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["S"]+SECTOR_IDXS["R"]) <= budgets[n]*0.15)
    elif p == 6:
        growth6 = SECTOR_IDXS["T"] + SECTOR_IDXS["C"] + SECTOR_IDXS["R"]
        model.addConstraint(xp.Sum(w[n][j] for j in growth6) <= budgets[n]*0.35)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["E"]+SECTOR_IDXS["F"]+SECTOR_IDXS["S"]) >= budgets[n]*0.40)
    elif p == 7:
        model.addConstraint(xp.Sum(w[n][j] for j in [tickers.index("AMZN"), tickers.index("TSLA")]) >= budgets[n]*0.30)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["T"]+SECTOR_IDXS["F"]+SECTOR_IDXS["H"]) <= budgets[n]*0.50)
    elif p == 8:
        model.addConstraint(xp.Sum(w[n][j] for j in GROWTH8) >= budgets[n]*0.50)
    elif p == 9:
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["S"]+SECTOR_IDXS["E"]) >= budgets[n]*0.20)
        model.addConstraint(xp.Sum(w[n][j] for j in SECTOR_IDXS["T"]+SECTOR_IDXS["C"]+SECTOR_IDXS["R"]) >= budgets[n]*0.30)

# -----------------------------
# 5) Helpers: randomize holdings & CI computation
# -----------------------------
rng_global = np.random.default_rng(RANDOM_SEED)

def randomize_holdings_dirichlet(base_holdings, budgets, alpha=DIRICHLET_ALPHA, rng=None):
    """
    base_holdings: (N × d) base dollar holdings
    budgets:       (N,)   dollar budgets
    Returns: (N × d) new holdings with small Dirichlet noise preserving budgets.
    """
    if rng is None:
        rng = rng_global
    N, d_ = base_holdings.shape
    assert d_ == d
    W = np.zeros_like(base_holdings, dtype=float)
    for n in range(N):
        w_base = base_holdings[n] / max(budgets[n], 1e-30)
        w_base = np.maximum(w_base, 1e-12)
        w_base = w_base / w_base.sum()
        # Dirichlet(alpha * w_base)
        a = alpha * w_base
        draw = rng.dirichlet(a)
        W[n] = draw * budgets[n]
    return W

def mean_ci(x, alpha=0.05):
    """
    Return mean, std, and (1-alpha) CI using normal approx: mean ± z * std/sqrt(n)
    """
    x = np.asarray(x, dtype=float)
    x = x[np.isfinite(x)]
    n = len(x)
    if n == 0:
        return np.nan, np.nan, np.nan, np.nan
    m = float(np.mean(x))
    s = float(np.std(x, ddof=1)) if n > 1 else 0.0
    z = 1.96  # ~95%
    half = z * s / np.sqrt(max(n, 1))
    return m, s, m - half, m + half

# -----------------------------
# 6) Core solver (build-time & solve-time separated)
# -----------------------------
def solve_infopool_once(N: int, tau_mat: np.ndarray,
                        budgets, mu_list, Sigma_list,
                        Omega_plus, Omega_minus,
                        old_holdings, theta_list):
    assert tau_mat.shape == (N, d)
    assert old_holdings.shape == (N, d)

    with xp.problem() as model:
        model.controls.outputlog = 0  # faster
        # model.controls.threads = -1  # optional

        tb0 = time.perf_counter()

        # variables
        w   = [[model.addVariable(lb=0.0, ub=budgets[n]) for j in range(d)] for n in range(N)]
        wp  = [[model.addVariable(lb=0.0, ub=budgets[n]) for j in range(d)] for n in range(N)]
        wm  = [[model.addVariable(lb=0.0, ub=budgets[n]) for j in range(d)] for n in range(N)]
        w_pct = [[model.addVariable(lb=0.0, ub=1.0)     for j in range(d)] for n in range(N)]

        # buy/sell split
        for n in range(N):
            for j in range(d):
                model.addConstraint(wp[n][j] - wm[n][j] == w[n][j] - old_holdings[n][j])

        # budgets + simplex
        for n in range(N):
            model.addConstraint(xp.Sum(w[n][j] for j in range(d)) == budgets[n])
            for j in range(d):
                model.addConstraint(w[n][j] == budgets[n] * w_pct[n][j])
            model.addConstraint(xp.Sum(w_pct[n][j] for j in range(d)) == 1)

        # constraints pattern
        for n in range(N):
            add_constraints_for_account(model, n, w, budgets)

        # objective
        lin = xp.Sum(mu_list[n][j] * w[n][j] for n in range(N) for j in range(d))
        risk = xp.Sum(
            -0.5 * theta_list[n] * (budgets[n]**2) *
            xp.Sum(Sigma_list[n][i][j] * w_pct[n][i] * w_pct[n][j] for i in range(d) for j in range(d))
            for n in range(N)
        )

        tau_arr = np.asarray(tau_mat, dtype=int)
        T_plus  = [xp.Sum(tau_arr[k, j] * wp[k][j]  for k in range(N)) for j in range(d)]
        T_minus = [xp.Sum(tau_arr[k, j] * wm[k][j] for k in range(N)) for j in range(d)]

        quad_pp = xp.Sum(T_plus[i]  * xp.Sum(Omega_plus[i,  j] * T_plus[j]  for j in range(d)) for i in range(d))
        quad_mm = xp.Sum(T_minus[i] * xp.Sum(Omega_plus[i,  j] * T_minus[j] for j in range(d)) for i in range(d))
        cross_pm= xp.Sum(T_plus[i]  * xp.Sum(Omega_minus[i, j] * T_minus[j] for j in range(d)) for i in range(d))
        impact_pool = 0.5 * (quad_pp + quad_mm + 2 * cross_pm)

        impact_ind = 0
        for m in range(N):
            r_p = [ (1 - tau_arr[m, j]) * wp[m][j]  for j in range(d) ]
            r_m = [ (1 - tau_arr[m, j]) * wm[m][j] for j in range(d) ]
            quad_pp_m = xp.Sum(r_p[i] * xp.Sum(Omega_plus[i,  j] * r_p[j] for j in range(d)) for i in range(d))
            quad_mm_m = xp.Sum(r_m[i] * xp.Sum(Omega_plus[i,  j] * r_m[j] for j in range(d)) for i in range(d))
            cross_pm_m= xp.Sum(r_p[i] * xp.Sum(Omega_minus[i, j] * r_m[j] for j in range(d)) for i in range(d))
            impact_ind += 0.5 * (quad_pp_m + quad_mm_m + 2.0 * cross_pm_m)

        impact = - ANNUAL_TRADING_DAYS * (impact_pool + impact_ind)
        model.setObjective(lin + risk + impact, sense=xp.maximize)

        tb1 = time.perf_counter()
        model.solve()
        ts1 = time.perf_counter()

        build_time = tb1 - tb0
        solve_time = ts1 - tb1

    return build_time, solve_time

# -----------------------------
# 7) Run scalability sweep with repeats: N = 2..50
# -----------------------------
runtime_long = []  # per-repeat records
summary_rows = []

TAU_FULL = TAU_BASE.copy()
rng = np.random.default_rng(RANDOM_SEED)

print("\n[info] Starting scalability sweep with repeats (N = 2..50, K = %d) ..." % REPEATS)
for N in range(2, N_MAX+1):
    budgets     = budgets_all[:N]
    mu_list     = mu_list_ALL[:N]
    Sigma_list  = Sigma_list_ALL[:N]
    theta_list  = theta_list_ALL[:N]
    tau_N       = TAU_FULL[:N, :].copy()

    
    base_holdings_N = old_holdings_ALL[:N, :]

    times_total = []
    for rep in range(REPEATS):
       
        seed_rep = rng.integers(0, 2**32-1, dtype=np.uint64)
        rng_rep = np.random.default_rng(int(seed_rep))
        noisy_holdings = randomize_holdings_dirichlet(
            base_holdings_N, budgets,
            alpha=DIRICHLET_ALPHA, rng=rng_rep
        )

        try:
            build_t, solve_t = solve_infopool_once(
                N, tau_N, budgets, mu_list, Sigma_list,
                Omega_plus, Omega_minus, noisy_holdings, theta_list
            )
            total_t = build_t + solve_t
            status = "ok"
        except Exception as e:
            build_t = np.nan; solve_t = np.nan; total_t = np.nan
            status = f"error: {e}"
            print(f"[ERR ] N={N}, rep={rep}: {e}")

        runtime_long.append({
            "N": N, "rep": rep,
            "build_time_sec": build_t,
            "solve_time_sec": solve_t,
            "total_time_sec": total_t,
            "status": status
        })
        if np.isfinite(total_t):
            times_total.append(total_t)

    m, s, lo, hi = mean_ci(times_total, alpha=0.05)
    summary_rows.append({
        "N": N,
        "mean_total_sec": m,
        "std_total_sec": s,
        "ci95_low": lo,
        "ci95_high": hi,
        "repeats_used": len(times_total)
    })
    print(f"[done] N={N}: mean={m:.4f}s, 95% CI=({lo:.4f},{hi:.4f}), used={len(times_total)}/{REPEATS}")

# Save long-form & summary
long_df = pd.DataFrame(runtime_long)
long_path = os.path.join(OUTDIR, "runtime_repeats_long.csv")
long_df.to_csv(long_path, index=False)
print("[info] Saved:", long_path)

summ_df = pd.DataFrame(summary_rows)
summ_path = os.path.join(OUTDIR, "runtime_summary.csv")
summ_df.to_csv(summ_path, index=False)
print("[info] Saved:", summ_path)

# -----------------------------
# 8) Visualization: mean curve + 95% CI ribbon
# -----------------------------
ok_df = summ_df[np.isfinite(summ_df["mean_total_sec"])].copy()
xN = ok_df["N"].values
y  = ok_df["mean_total_sec"].values
lo = ok_df["ci95_low"].values
hi = ok_df["ci95_high"].values

plt.figure(figsize=(7.6, 4.6))
plt.plot(xN, y, marker="o", label="Mean total time")
plt.fill_between(xN, lo, hi, alpha=0.25, label="95% CI")
plt.xlabel("Number of Accounts (N)")
plt.ylabel("Total Time (seconds)")
plt.title("InfoPool Single-Period Scalability with Uncertainty (Assets = 20): N=2..50")
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend()
png_path = os.path.join(OUTDIR, "runtime_vs_accounts_CI.png")
plt.tight_layout()
plt.savefig(png_path, dpi=180)
plt.close()
print("[info] Plot saved:", png_path)

# -----------------------------
# 9) log-log regression: slope & 95% CI (analytical)
# -----------------------------
# 只对正值做回归
mask = (y > 0) & (xN > 0)
x = np.log(xN[mask])
t = np.log(y[mask])

# OLS: t = a + b * x + eps
X = np.vstack([np.ones_like(x), x]).T
beta_hat = np.linalg.lstsq(X, t, rcond=None)[0]  # [a, b]
resid = t - X @ beta_hat
n, k = X.shape
sigma2 = (resid @ resid) / (n - k)
cov_beta = sigma2 * np.linalg.inv(X.T @ X)
se_b = np.sqrt(cov_beta[1, 1])
b = float(beta_hat[1])
# 95% CI for slope using normal approx
b_lo = b - 1.96 * se_b
b_hi = b + 1.96 * se_b

# 画图
xx = np.linspace(x.min(), x.max(), 200)
tt = beta_hat[0] + beta_hat[1] * xx
plt.figure(figsize=(7.0, 4.4))
plt.scatter(x, t, s=20, alpha=0.7, label="Data (log-log)")
plt.plot(xx, tt, lw=2, label=f"OLS fit: slope={b:.3f} (95% CI [{b_lo:.3f},{b_hi:.3f}])")
plt.xlabel("log N")
plt.ylabel("log Time")
plt.title("Log–Log Fit of Runtime vs. N")
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend()
logfit_path = os.path.join(OUTDIR, "loglog_fit.png")
plt.tight_layout()
plt.savefig(logfit_path, dpi=180)
plt.close()
print("[info] Plot saved:", logfit_path)

# 附加写入回归结果
with open(os.path.join(OUTDIR, "scalability_regression.txt"), "w", encoding="utf-8") as f:
    f.write(f"log-log OLS slope (time vs N): {b:.6f}\n")
    f.write(f"95% CI: [{b_lo:.6f}, {b_hi:.6f}]\n")
    f.write(f"n={n}, residual variance={sigma2:.6e}\n")
print("[info] Regression summary saved.")


[info] Preparing market data and impact matrices (pickle cache) ...

[info] Starting scalability sweep with repeats (N = 2..50, K = 10) ...
[done] N=2: mean=0.0592s, 95% CI=(0.0528,0.0657), used=10/10
[done] N=3: mean=0.0662s, 95% CI=(0.0636,0.0689), used=10/10
[done] N=4: mean=0.0846s, 95% CI=(0.0767,0.0925), used=10/10
[done] N=5: mean=0.0991s, 95% CI=(0.0964,0.1018), used=10/10
[done] N=6: mean=0.1164s, 95% CI=(0.1078,0.1251), used=10/10
[done] N=7: mean=0.1251s, 95% CI=(0.1220,0.1281), used=10/10
[done] N=8: mean=0.1476s, 95% CI=(0.1438,0.1514), used=10/10
[done] N=9: mean=0.1823s, 95% CI=(0.1729,0.1917), used=10/10
[done] N=10: mean=0.2243s, 95% CI=(0.2064,0.2421), used=10/10
[done] N=11: mean=0.2340s, 95% CI=(0.2242,0.2438), used=10/10
[done] N=12: mean=0.2837s, 95% CI=(0.2688,0.2987), used=10/10
[done] N=13: mean=0.2943s, 95% CI=(0.2832,0.3054), used=10/10
[done] N=14: mean=0.3575s, 95% CI=(0.3465,0.3685), used=10/10
[done] N=15: mean=0.3844s, 95% CI=(0.3752,0.3935), used=10/10


In [7]:
# -*- coding: utf-8 -*-
"""
Efficient frontier (single-asset points, 20 names)
- Dominated assets: hollow markers
- Non-dominated assets: solid markers
- Prints, for each asset, whether it is dominated and by which assets
- Saves PNG + CSV to infopool_batch_outputs/
"""


# -----------------------------
# 0) Paths & constants
# -----------------------------
OUTDIR = "infopool_batch_outputs"
os.makedirs(OUTDIR, exist_ok=True)

ANNUAL_TRADING_DAYS = 252

TICKERS = [
    "AAPL","MSFT","GOOGL","AMZN","META","TSLA","JNJ","JPM","V","PG",
    "NVDA","DIS","UNH","HD","MA","PFE","BAC","KO","XOM","CVX"
]
START_DATE = "2020-01-01"
END_DATE   = "2025-07-24"

PNG_PATH = os.path.join(OUTDIR, "assets_efficient_frontier_20.png")
CSV_PATH = os.path.join(OUTDIR, "assets_eff_summary_20.csv")

# -----------------------------
# 1) Download prices and build daily returns
#    NOTE: auto_adjust=False so 'Adj Close' is available.
#          Fallback to 'Close' if 'Adj Close' is missing.
# -----------------------------
data = yf.download(
    TICKERS, start=START_DATE, end=END_DATE,
    auto_adjust=False, progress=False
)

if isinstance(data.columns, pd.MultiIndex):
    lvl0 = data.columns.get_level_values(0)
    if "Adj Close" in set(lvl0):
        prices = data["Adj Close"]
    elif "Close" in set(lvl0):
        prices = data["Close"]
    else:
        raise KeyError(f"No 'Adj Close' or 'Close' in columns. Top-level fields found: {sorted(set(lvl0))}")
else:
    # Rare case: single-level columns; try to pick a sensible field
    if "Adj Close" in data.columns:
        prices = data[["Adj Close"]]
    elif "Close" in data.columns:
        prices = data[["Close"]]
    else:
        raise KeyError(f"No 'Adj Close' or 'Close' in columns: {list(data.columns)}")

prices = prices.dropna(how="all")
returns = prices.pct_change().dropna()

# If single-level (one column) because of unexpected format, reshape to wide with tickers.
if returns.shape[1] == 1 and len(TICKERS) > 1:
    # Last-resort: refetch with auto_adjust=True and columns=tickers (yfinance sometimes flattens)
    data2 = yf.download(TICKERS, start=START_DATE, end=END_DATE, auto_adjust=True, progress=False)
    if isinstance(data2.columns, pd.MultiIndex) and "Close" in set(data2.columns.get_level_values(0)):
        prices2 = data2["Close"]
    else:
        # With auto_adjust=True, 'Close' is adjusted; for multi-index it’s usually 'Close'
        # For single-level, columns should be tickers already
        prices2 = data2.copy()
    prices2 = prices2.dropna(how="all")
    returns = prices2.pct_change().dropna()

# -----------------------------
# 2) Annualized mean (μ) and volatility (σ) per asset
# -----------------------------
mu_daily = returns.mean()
sig_daily = returns.std()

mu_annual = mu_daily * ANNUAL_TRADING_DAYS
sig_annual = sig_daily * np.sqrt(ANNUAL_TRADING_DAYS)

df = pd.DataFrame({
    "ticker": mu_annual.index,
    "mu_annual": mu_annual.values,
    "sigma_annual": sig_annual.values
}).set_index("ticker")

# -----------------------------
# 3) Dominance test among single-asset points
#    i is dominated if ∃ j: sigma_j <= sigma_i and mu_j >= mu_i (with ≥ in both and at least one strict)
# -----------------------------
dominators = {t: [] for t in df.index}
for i in df.index:
    mu_i, s_i = df.loc[i, "mu_annual"], df.loc[i, "sigma_annual"]
    for j in df.index:
        if j == i:
            continue
        mu_j, s_j = df.loc[j, "mu_annual"], df.loc[j, "sigma_annual"]
        weak = (s_j <= s_i) and (mu_j >= mu_i)
        strict = (s_j < s_i) or (mu_j > mu_i)
        if weak and strict:
            dominators[i].append(j)

df["dominated"]  = [len(dominators[t]) > 0 for t in df.index]
df["dominators"] = [dominators[t] for t in df.index]

# Upper envelope (optional)
nd_df = df[~df["dominated"]].copy().sort_values("sigma_annual")
frontier = []
last_mu = -np.inf
for t, row in nd_df.iterrows():
    if row["mu_annual"] > last_mu + 1e-15:
        frontier.append((row["sigma_annual"], row["mu_annual"]))
        last_mu = row["mu_annual"]
frontier = np.array(frontier)

# -----------------------------
# 4) Plot
# -----------------------------
plt.figure(figsize=(7.8, 5.0))

# Dominated: hollow markers
dom = df[df["dominated"]]
plt.scatter(
    dom["sigma_annual"], dom["mu_annual"],
    s=64, facecolors="none", edgecolors="black", linewidths=1.5, label="Dominated"
)

# Non-dominated: solid markers
nd = df[~df["dominated"]]
plt.scatter(
    nd["sigma_annual"], nd["mu_annual"],
    s=64, color="black", label="Non-dominated"
)

# Frontier line
if frontier.size >= 2:
    plt.plot(frontier[:, 0], frontier[:, 1], "-", linewidth=1.6, color="black", alpha=0.85)

# Light labels for non-dominated only
for t, row in nd.iterrows():
    plt.annotate(t, (row["sigma_annual"], row["mu_annual"]),
                 xytext=(4, 4), textcoords="offset points", fontsize=9)

plt.title("Single-Asset Efficient Frontier (Annualized, 20 Assets)")
plt.xlabel("Volatility σ (annualized)")
plt.ylabel("Expected Return μ (annualized)")
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend(loc="lower right")
plt.tight_layout()
plt.savefig(PNG_PATH, dpi=200)
plt.close()

# -----------------------------
# 5) Save CSV + print dominance report
# -----------------------------
df_out = df.copy()
df_out["dominators"] = df_out["dominators"].apply(lambda L: ",".join(L) if len(L) else "")
df_out.to_csv(CSV_PATH, float_format="%.6f")

print("Dominance report (single-asset points):")
for t in df.index:
    if df.loc[t, "dominated"]:
        print(f"- {t}: DOMINATED by {', '.join(dominators[t])}")
    else:
        print(f"- {t}: NOT dominated")

print(f"\nSaved plot: {PNG_PATH}")
print(f"Saved table: {CSV_PATH}")


Dominance report (single-asset points):
- AAPL: DOMINATED by MSFT
- AMZN: DOMINATED by AAPL, GOOGL, MSFT
- BAC: DOMINATED by AAPL, CVX, GOOGL, HD, JPM, MA, MSFT, V, XOM
- CVX: DOMINATED by AAPL, GOOGL, HD, JPM, MA, MSFT, V, XOM
- DIS: DOMINATED by AAPL, GOOGL, HD, JNJ, JPM, KO, MA, MSFT, PG, UNH, V, XOM
- GOOGL: DOMINATED by AAPL, MSFT
- HD: NOT dominated
- JNJ: NOT dominated
- JPM: DOMINATED by MSFT
- KO: NOT dominated
- MA: DOMINATED by MSFT
- META: NOT dominated
- MSFT: NOT dominated
- NVDA: NOT dominated
- PFE: DOMINATED by JNJ, KO, PG
- PG: DOMINATED by KO
- TSLA: DOMINATED by NVDA
- UNH: DOMINATED by AAPL, GOOGL, HD, JNJ, JPM, KO, MA, MSFT, PG, V
- V: NOT dominated
- XOM: DOMINATED by AAPL, GOOGL, JPM, MSFT

Saved plot: infopool_batch_outputs\assets_efficient_frontier_20.png
Saved table: infopool_batch_outputs\assets_eff_summary_20.csv
